In [4]:
# импортируем всё необходимое с учетом настроек с семинара
import pandas as pd
import seaborn as sn
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
import os
from openpyxl import load_workbook
from openpyxl.styles import Border, Side, Font
# Вывод таблицы в удобном формате
from tabulate import tabulate
from scipy.stats import norm, t, chi2
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
np.set_printoptions(legacy='1.21')
np.set_printoptions(suppress=True)
import matplotlib.pyplot as pt
pt.rcParams["font.size"] = 14
pt.rcParams['figure.figsize'] = (15, 8)
sn.set_style("whitegrid")

## Задача 1

#### 1. Чтение файла

In [5]:
child = pd.read_csv(r'C:\Users\admin\metrix1-2024-hw4\fertil2.txt', delimiter = '\t', na_values = '.') # учитываем, что файл с расширением txt
child

,mnthborn,yearborn,age,electric,radio,tv,bicycle,educ,ceb,agefbrth,children,knowmeth,usemeth,monthfm,yearfm,agefm,idlnchld,heduc,agesq,urban,urb_educ,spirit,protest,catholic,frsthalf,educ0,evermarr,educ7
0,5,64,24,1.000,1.000,1.000,1.000,12,0,NaN,0,1.000,0.000,NaN,NaN,NaN,2.000,NaN,576,1,12,0,0,0,1,0,0,1
1,1,56,32,1.000,1.000,1.000,1.000,13,3,25.000,3,1.000,1.000,11.000,80.000,24.000,3.000,12.000,1024,1,13,0,0,0,1,0,1,1
2,7,58,30,1.000,0.000,0.000,0.000,5,1,27.000,1,1.000,0.000,6.000,83.000,24.000,5.000,7.000,900,1,5,1,0,0,0,0,1,0
3,11,45,42,1.000,0.000,1.000,0.000,4,3,17.000,2,1.000,0.000,1.000,61.000,15.000,3.000,11.000,1764,1,4,0,0,0,0,0,1,0
4,5,45,43,1.000,1.000,1.000,1.000,11,2,24.000,2,1.000,1.000,3.000,66.000,20.000,2.000,14.000,1849,1,11,0,1,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4356,6,72,16,0.000,1.000,0.000,0.000,7,0,NaN,0,1.000,0.000,NaN,NaN,NaN,2.000,NaN,256,0,0,0,1,0,1,0,0,1
4357,1,60,28,0.000,1.000,0.000,0.000,7,2,19.000,2,1.000,1.000,NaN,NaN,NaN,4.000,NaN,784,0,0,0,1,0,1,0,0,1
4358,11,63,24,0.000,1.000,0.000,0.000,5,4,13.000,3,1.000,1.000,NaN,NaN,NaN,4.000,NaN,576,0,0,0,1,0,0,0,0,0
4359,1,62,26,0.000,1.000,0.000,0.000,0,1,25.000,1,1.000,0.000,5.000,84.000,22.000,5.000,7.000,676,0,0,1,0,0,1,1,1,0


In [6]:
child['agesq'] = child['age'] ** 2
#child

#### 2. Вывод суммарных статистик и их сохранение в таблицу эксель

In [7]:
df = child[['educ', 'age', 'children', 'agesq']]
df = df.describe().transpose() # транспонируем полученную табличку, чтобы было красивее
df

,count,mean,std,min,25%,50%,75%,max
educ,4361.000,5.856,3.927,0.000,3.000,7.000,8.000,20.000
age,4361.000,27.405,8.685,15.000,20.000,26.000,33.000,49.000
children,4361.000,2.268,2.222,0.000,0.000,2.000,4.000,13.000
agesq,4361.000,826.460,526.923,225.000,400.000,676.000,1089.000,2401.000


На всякий случай добавила квадрат возраста, тк эта переменная используется в модели, хотя большого смысла включать ее в таблицу описательных характеристик в принципе нет

In [8]:
def save_to_excel(file, name):
    folder = r'C:\Users\admin\metrix1-2024-hw4' # задаем сразу папку с дз, куда будут сохраняться все файлы
    filepath = os.path.join(folder, name)
    if not name.endswith('.xlsx'):
        filepath += '.xlsx' # добавляем к имени файла нужное расширение
    try:
        file.to_excel(filepath) # сохраняем файл по указанному пути в папку
        wb = load_workbook(filepath) # загружаем сохраненную книгу эксель для дальнейшей работы с ней
        ws = wb.active
        thin = Side(border_style="thin", color="000000") # далее настраиваем прочерчивание границ в таблице
        border = Border(left=thin, right=thin, top=thin, bottom=thin)
        for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
            for cell in row:
                cell.border = border
        wb.save(filepath) # сохраняем изменения по указанному пути
        print(f'Таблица сохранена под именем {name}')
    except Exception as e:
        print(f'Произошла ошибка при сохранении: {e}') # выводим описание ошибки в удобном читаемом виде, если что-то пошло не так

In [9]:
save_to_excel(df, 'description')

Таблица сохранена под именем description


#### Оценка модели

In [10]:
y = child['children'] # задаем зависимую переменную
X = sm.add_constant(child[['educ', 'age', 'agesq']]) # добавляем технический коэффициент (константу) к регрессору

reg_children = sm.OLS(y, X).fit(cov_type='HC0') # задаем гетероскедастичность для ошибок (std errors) (по умолчанию была бы гомоскедастичность)
reg_children.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               children   R-squared:                       0.569
Model:                            OLS   Adj. R-squared:                  0.568
Method:                 Least Squares   F-statistic:                     1924.
Date:                Fri, 13 Dec 2024   Prob (F-statistic):               0.00
Time:                        13:49:51   Log-Likelihood:                -7835.6
No. Observations:                4361   AIC:                         1.568e+04
Df Residuals:                    4357   BIC:                         1.570e+04
Df Model:                           3                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.1383      0.244    -16.994      0.000      -4.616      -3.661
educ          -0.0906      0.006    -14.982      0.000      -0.102      -0.079
age            0.3324      0.019     17.317      0.000       0.295       0.370
agesq         -0.0026      0.000     -7.478      0.000      -0.003      -0.002
==============================================================================
Omnibus:                      203.406   Durbin-Watson:                   1.868
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              715.951
Skew:                           0.017   Prob(JB):                    3.41e-156
Kurtosis:                       4.985   Cond. No.                     1.07e+04
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.07e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
def save_regression_results(results, name):
    model_names = ['Число выживших детей']
    data = {}
    folder = r'C:\Users\admin\metrix1-2024-hw4'
    filepath = os.path.join(folder, name)
    if not name.endswith('.xlsx'):
        filepath += '.xlsx'
    if not isinstance(results, (list, tuple)):
        results = [results]
    for model_name, result in zip(model_names, results): # передаем этим переменным нужные нам значения из анализа регрессии
        params = result.params
        conf = result.bse
        pvalues = result.pvalues
        r2 = result.rsquared
        nobs = int(result.nobs)

        model_data = {}
        for var in ['const', 'educ', 'age', 'agesq']:
            if var in params:
                signif = "***" if pvalues[var] < 0.01 else ("**" if pvalues[var] < 0.05 else ("*" if pvalues[var] < 0.1 else "")) # задаем параметры из задания для звездочек
                model_data[var] = f"{params[var]:.3f}{signif} ({conf[var]:.3f})" # выводим ошибку в скобках
            else:
                model_data[var] = "-"

        model_data["Observations"] = f"{nobs}" # добавляем число наблюдений и R^2
        model_data["R^2"] = f"{r2:.3f}"

        data[model_name] = model_data

    df = pd.DataFrame(data) # переводим в датафрейм

    df = df.loc[['educ', 'age', 'agesq', 'const', 'Observations', 'R^2']]
    
    df.to_excel(filepath) # сохраняем в эксель

    wb = load_workbook(filepath)
    ws = wb.active

    new_row_names = ['число лет образования', 'возраст', 'квадрат возраста', 'константа', 'Число наблюдений', 'R^2'] # переименовываем строки
    for row_num, new_name in enumerate(new_row_names, start=2): 
        ws.cell(row=row_num, column=1, value=new_name)

    thin = Side(border_style="thin", color="000000") # прочерчиваем границы и задаем их ширину
    border = Border(left=thin, right=thin, top=thin, bottom=thin)
    for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
        for cell in row:
            cell.border = border
    for column in ws.columns:
        max_length = 0
        column_letter = column[0].column_letter 
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except Exception as e:
                pass
        adjusted_width = (max_length + 2) 
        ws.column_dimensions[column_letter].width = adjusted_width

    wb.save(filepath)

    print(f'Таблица сохранена под именем "{name}"')

In [12]:
save_regression_results(reg_children, 'регрессия')

Таблица сохранена под именем "регрессия"


#### 4. Статистическая значимость коэффициентов

In [13]:
regressor_names = reg_children.params.index.to_list()
coefficients = reg_children.params
standard_errors = reg_children.bse
t_statistics = reg_children.tvalues
p_values = reg_children.pvalues

print(p_values)

const   0.000
educ    0.000
age     0.000
agesq   0.000
dtype: float64


In [14]:
alpha = 0.05
z_crit = 1.96
stat_significance_df = pd.Series()

for regressor_name, z in t_statistics.items():
    is_significant = np.abs(z) > z_crit # critical rule
    stat_significance_df[regressor_name] = is_significant

print('Following table shows whether regressor is significant or not at 5% level')
display(stat_significance_df)

Following table shows whether regressor is significant or not at 5% level


const    True
educ     True
age      True
agesq    True
dtype: bool

Итак, мы не можем отвергнуть нулевую гипотезу о статистической незначимости какого-либо коэффициента данной регресии на уровне значимости 5%

#### 5. Зависимость числа детей от образования

In [15]:
print(coefficients)

const   -4.138
educ    -0.091
age      0.332
agesq   -0.003
dtype: float64


Интерпретация прогноза. В среднем при прочих равных условиях для двух наблюдений по числу лет образования, отличающихся на 1 год, соответствующее данным наблюдениям число детей будет отличаться на 0,091 ребенка (зависимость обратная).
С точки зрения причинно-следственных связей не можем интерпретировать, тк переменная эндогенная (будет рассмотрено далее)

#### 6. Зависимость числа детей от возраста

In [16]:
b1 = reg_children.params['agesq']
b1

-0.0026308230574148134

In [17]:
round(b1*2, 3)

-0.005

Интерпретация прогноза. В среднем при прочих равных условиях при сравнении двух наблюдений с одинаковым числом лет образования, отличающихся на дельту по возрасту (маленькое), среднее число выживших детей, им соответствующее, будет отличаться на (0,332-0,005*age)*дельта.

Объяснение из экономических соображений: до определенного возраста чем старше человек, тем скорее всего у него больше детей, тем большее число и выживших детей, а после определенного возраста дети уже начяинают умирать от естественных причин, а женщины из наблюдений перестают рожать => тогда общий эффект увеличения возраста даст здесь отрицательное совокупное влияние на число выживших детей. Поэтому здесь также можно интерпретировать как причинно-следственную связь: в среднем при прочих равных условиях при увеличении возраста на малое дельта число выживших детей увеличится на (0,332-0,005*age)*дельта

Включение в модель квадрата возраста помогает справиться с немонотонной зависимостью числа выживших детей от возраста наблюдаемого в среднем при прочих равных. Парабола получилась ветвями вниз, то есть как я уже и обосновала логически, сначала увеличение возраста дает большой прирост к числу выживших детей, потом в вершине параболы достигает своего пика и затем убывает. Нелинейная зависимость в регрессии позволяет нам всё это учесть.

#### 7. Reverse causality: влияние числа детей на образование

В данном случае регрессор и зависимую переменную действительно можно уличить во взаимном влиянии друг на друга. и если первое мы уже рассмотрели выше, то второе касается того, что чем больше у объекта наблюдений детей (и чем раньше они у него рождаются), тем скорее всего меньше времени он может позволить себе потратить на образование в среднем при прочих равных условиях. К тому же, детей надо содержать, обеспечивать семью, то есть работать, а не учиться. Получается, чем больше х, тем меньше у, тем еще больше х => еще больше у и тп. И тогда бета1 по модулю будет меньше, чем если бы не было этого взаимного влияния educ и children, то есть по модулю эффект влияния образования на число детей будет недооцениваться.

#### 8. Переменная frsthalf: можно ли сделать вывод об экзогенности с экономической точки зрения?

В каком полугодии родилась женщина - едва ли как-либо влияет на число ее выживших детей, поэтому скорее всего это незначимая переменная, если бы мы ее добавили в регрессию, то есть она не будет скоррелирована с ошибкой (мы не теряем точность модели, если не добавляем эту переменную в регрессию, тк она никак не объясняет children через какие-либо пропущенные переменные с точки зрения экономики)

#### 9. frsthalf - инструмент?

In [18]:
# проверка корреляции с educ (фактически проверяем силу инструмента тестов Вальда). Первый шаг 2SLS
y = child[['educ']]
X = child[['frsthalf', 'age', 'agesq']]
X = sm.add_constant(X)

reg_educ = sm.OLS(y, X)
reg_educ = reg_educ.fit(cov_type = 'HC0')

reg_educ.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   educ   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     201.9
Date:                Fri, 13 Dec 2024   Prob (F-statistic):          1.30e-122
Time:                        13:49:52   Log-Likelihood:                -11905.
No. Observations:                4361   AIC:                         2.382e+04
Df Residuals:                    4357   BIC:                         2.384e+04
Df Model:                           3                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          9.6929      0.541     17.910      0.000       8.632      10.754
frsthalf      -0.8523      0.113     -7.528      0.000      -1.074      -0.630
age           -0.1080      0.040     -2.685      0.007      -0.187      -0.029
agesq         -0.0005      0.001     -0.744      0.457      -0.002       0.001
==============================================================================
Omnibus:                       51.991   Durbin-Watson:                   1.352
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               56.555
Skew:                           0.233   Prob(JB):                     5.24e-13
Kurtosis:                       3.306   Cond. No.                     1.05e+04
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.05e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [19]:
reg_educ.wald_test('frsthalf = 0', scalar=True)

<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=56.67161443935808, p-value=5.150163785199053e-14, df_denom=1>

56 > 10*1 => согласно Rule of thumb инструмент не слабый, то есть не отвергаем нулевую гипотезу на уровне значимоси 5% о плохой корреляции frsthalf с educ
С экономической точки зрения это объясняется тем, что при прочих равных условиях (то есть с равным возрастом) в среднем у двух женщин число лет образования будет выше, если она родилась в первой половине года, поскольку в таком случае больше вероятность, что она пойдет в школу в том году, когда ей исполнится 7 лет. Вторая женщина такого же возраста, но рожденная во второй половине года, скорее всего пойдет в школу в тот год, когда ей исполнится уже 8 лет, и тогда при их сравнении у первой будет образование в среднем на год больше даже при их одинаковом возрасте

Итак, мы проверили два условия для того, чтобы frsthalf использовать в качестве инструмента для educ: экзогенность (отсутствие корреляции с ошибкой регрессии) и релевантность (корреляция с educ, проверили смысловую значимость и статистическую тестом Вальда). Также есть еще несколько условий применимости 2SLS: случайность выборки (делаем такое предположение в наших данных), существование 4-ых моментов у всех величин (выбросы маловероятны), экзогенность других переменных (age, agesq - по смыслу они действительно экзогенны) => тогда оценки по 2SLS будут состоятельными

Итак, при фиксированном возрасте (и соответственно квадрате возраста) frsthalf влияет на children через educ (не непосредственно). Тогда мы действительно можем применять frsthalf в качестве инструмента для educ. Также должно выполняться условие: число инструментов >= числа эндогенных переменных. 1 >= 1 - верно

#### 10. 2SLS

In [20]:
!pip install linearmodels

In [22]:
from linearmodels.iv.model import IV2SLS
y = child[['children']]
W = child[['age', 'agesq']]
z = child[['frsthalf']]
X_endog = child[['educ']]
W = sm.add_constant(W)

model = IV2SLS(dependent=y, exog=W, endog=X_endog, instruments=z)
results_2SLS = model.fit(cov_type='robust')
results_2SLS.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:               children   R-squared:                      0.5502
Estimator:                    IV-2SLS   Adj. R-squared:                 0.5499
No. Observations:                4361   F-statistic:                    5520.3
Date:                Fri, Dec 13 2024   P-value (F-stat)                0.0000
Time:                        13:50:37   Distribution:                  chi2(3)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -3.3878     0.5449    -6.2168     0.0000     -4.4559     -2.3197
age            0.3236     0.0202     15.998     0.0000      0.2840      0.3633
agesq         -0.0027     0.0004    -7.5873     0.0000     -0.0034     -0.0020
educ          -0.1715     0.0524    -3.2753     0.0011     -0.2741     -0.0689
==============================================================================

Endogenous: educ
Instruments: frsthalf
Robust Covariance (Heteroskedastic)
Debiased: False
"""

#### 11. Добавляем новые пременные

In [30]:
child_new = child.dropna(subset=['electric', 'tv', 'bicycle'])
#child_new

In [31]:
y = child_new['children'] # задаем зависимую переменную
X = sm.add_constant(child_new[['educ', 'age', 'agesq', 'electric', 'tv', 'bicycle']]) # добавляем технический коэффициент (константу) к регрессору

reg_bi = sm.OLS(y, X).fit(cov_type='HC0') # задаем гетероскедастичность для ошибок (std errors) (по умолчанию была бы гомоскедастичность)
reg_bi.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               children   R-squared:                       0.576
Model:                            OLS   Adj. R-squared:                  0.575
Method:                 Least Squares   F-statistic:                     974.1
Date:                Fri, 13 Dec 2024   Prob (F-statistic):               0.00
Time:                        13:57:49   Log-Likelihood:                -7789.3
No. Observations:                4356   AIC:                         1.559e+04
Df Residuals:                    4349   BIC:                         1.564e+04
Df Model:                           6                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.3898      0.244    -17.973      0.000      -4.868      -3.911
educ          -0.0767      0.006    -12.101      0.000      -0.089      -0.064
age            0.3402      0.019     17.797      0.000       0.303       0.378
agesq         -0.0027      0.000     -7.750      0.000      -0.003      -0.002
electric      -0.3027      0.074     -4.073      0.000      -0.448      -0.157
tv            -0.2531      0.083     -3.065      0.002      -0.415      -0.091
bicycle        0.3179      0.049      6.498      0.000       0.222       0.414
==============================================================================
Omnibus:                      196.639   Durbin-Watson:                   1.891
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              675.502
Skew:                           0.009   Prob(JB):                    2.07e-147
Kurtosis:                       4.929   Cond. No.                     1.08e+04
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 1.08e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [32]:
y = child_new[['children']]
W = child_new[['age', 'agesq', 'electric', 'tv', 'bicycle']]
z = child_new[['frsthalf']]
X_endog = child_new[['educ']]
W = sm.add_constant(W)

model = IV2SLS(dependent=y, exog=W, endog=X_endog, instruments=z)
results_2SLS = model.fit(cov_type='robust')
results_2SLS.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:               children   R-squared:                      0.5577
Estimator:                    IV-2SLS   Adj. R-squared:                 0.5571
No. Observations:                4356   F-statistic:                    5643.9
Date:                Fri, Dec 13 2024   P-value (F-stat)                0.0000
Time:                        13:59:53   Distribution:                  chi2(6)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -3.5913     0.6389    -5.6213     0.0000     -4.8435     -2.3391
age            0.3281     0.0213     15.399     0.0000      0.2864      0.3699
agesq         -0.0027     0.0004    -7.7774     0.0000     -0.0034     -0.0020
electric      -0.1065     0.1582    -0.6733     0.5008     -0.4167      0.2036
tv            -0.0026     0.2043    -0.0125     0.9900     -0.4029      0.3978
bicycle        0.3321     0.0506     6.5572     0.0000      0.2328      0.4313
educ          -0.1640     0.0643    -2.5491     0.0108     -0.2901     -0.0379
==============================================================================

Endogenous: educ
Instruments: frsthalf
Robust Covariance (Heteroskedastic)
Debiased: False
"""

Оцениваем с точки зрения причинно-следственных связей, тк для прогноза инструменты обычно не используют (ошибки больше)
В модели без инструмента в среднем при прочих равных условиях при увеличении образования на 1 год число выживших детей уменьшается на 0,0767. В модели с инструментов в среднем при прочих равных условиях при увеличении образования на 1 год число выживших детей уменьшается на 0,164. Таким образом, в модели без инструмента эффект по модулю был действительно занижен, как мы и говорили. В модели с инструментом эффект по модулю выше, его оценка более справедллива, поэтому предпочитаем вторую модель (то есть с инструментом).

#### 12. Коэффициент при tv

Во второй модели, которую мы выбрали, мы не можем отвергнуть нулевую гипотезу о статистической незначимости данной переменной даже на уровне значимости 10% (Z = 1,64). Поэтому данный регрессор незначим статистически.
Однако для прогноза лучше использовать OLS, поэтому: в среднем при прочих равных условиях при сравнении двух женщин с одинаковыми educ, age, agesq, electric, bicycle, но отличающихся наличием телевизора в доме, та женщина, у которой есть телевизор, будет иметь в среднем на 0,2531 меньше выживших детей, чем та, у которой нет